## WeGo Data Introduction

In [1]:
import pandas as pd

In [2]:
wego = pd.read_csv("../data/Route 50 Timepoint and Headway Data, 1-1-2023 through 5-12-2025.csv")

All the data that you have been provided is from Route 50, Charlotte Pike.

In [3]:
wego['ROUTE_ABBR'].value_counts().sort_index()

ROUTE_ABBR
50    618998
Name: count, dtype: int64

The trip can be identified by the DATE/CALENDAR_ID plus the TRIP_ID.  
**Warning:** The TRIP_ID refers to the route and time but will be used across multiple days.

The data contains multiple **time points** for each trip. There are more stops along the route than time points, but the time points are the points with specific scheduled times the bus operators must adhere to.

The first stop of a trip has a TRIP_EDGE of 1, the last has a TRIP_EDGE of 2, and the intermediate stops are TRIP_EDGE 0. 

Here is the first trip in the dataset. It occurred on January 1, 2023 and was scheduled to start at 5:34 and end at 6:05.

In [4]:
(
    wego
    .loc[wego['CALENDAR_ID'] == 120230101]
    .loc[wego['TRIP_ID'] == 332422]
    [['DATE', 'CALENDAR_ID', 'TRIP_ID', 'ROUTE_ABBR', 'TIME_POINT_ABBR', 'TRIP_EDGE', 'SCHEDULED_TIME']]
)

,DATE,CALENDAR_ID,TRIP_ID,ROUTE_ABBR,TIME_POINT_ABBR,TRIP_EDGE,SCHEDULED_TIME
0,2023-01-01,120230101,332422,50,WALM,1,05:34:00
1,2023-01-01,120230101,332422,50,HLWD,0,05:40:00
2,2023-01-01,120230101,332422,50,WHBG,0,05:47:00
3,2023-01-01,120230101,332422,50,CH46,0,05:50:00
4,2023-01-01,120230101,332422,50,28&CHARL,0,05:54:00
5,2023-01-01,120230101,332422,50,MCC5_1,2,06:05:00


Note that the same TRIP_ID appears on the following day with a different CALENDAR_ID.

In [5]:
(
    wego
    .loc[wego['CALENDAR_ID'] == 120230102]
    .loc[wego['TRIP_ID'] == 332422]
    [['DATE', 'CALENDAR_ID', 'TRIP_ID', 'ROUTE_ABBR', 'TIME_POINT_ABBR', 'TRIP_EDGE', 'SCHEDULED_TIME']]
)

,DATE,CALENDAR_ID,TRIP_ID,ROUTE_ABBR,TIME_POINT_ABBR,TRIP_EDGE,SCHEDULED_TIME
516,2023-01-02,120230102,332422,50,WALM,1,05:34:00
517,2023-01-02,120230102,332422,50,HLWD,0,05:40:00
518,2023-01-02,120230102,332422,50,WHBG,0,05:47:00
519,2023-01-02,120230102,332422,50,CH46,0,05:50:00
520,2023-01-02,120230102,332422,50,28&CHARL,0,05:54:00
521,2023-01-02,120230102,332422,50,MCC5_1,2,06:05:00


**Adherence** refers to the difference between scheduled time and the actual time that the bus departs from a stop.

A negative value for ADHERENCE indicates that the bus is late, and a positive indicates that the bus is early.

Generally, an adherence value less than -6 is considered late, and greater than 1 is considered early, but there are some exceptions. For example, a positive adherence for the end of a trip (TRIP_EDGE 2) is not considered early, since it is not a problem if a bus ends its trip early as long as it didn't pass other timepoints early along the way. You can check whether a trip was considered on-time, early, or late using the ADJUSTED_EARLY_COUNT, ADJUSTED_LATE_COUNT, and ADJUSTED_ONTIME_COUNT columns.

Here is an example of a trip where all time points would be considered to be on time. Notice that at the end of the trip, the bus was almost 3 minutes early, but was still counted as on-time since this was a trip edge of 2. Also, the bus departed more than 5 minutes late but was still considered to be on-time.

In [6]:
(
    wego
    .loc[wego['CALENDAR_ID'] == 120230101]
    .loc[wego['TRIP_ID'] == 332423]
    [[
        'DATE', 'CALENDAR_ID', 'TRIP_ID', 'ROUTE_ABBR',
        'TIME_POINT_ABBR', 'TRIP_EDGE',
        'SCHEDULED_TIME', 'ACTUAL_DEPARTURE_TIME', 'ADHERENCE',
        'ADJUSTED_EARLY_COUNT', 'ADJUSTED_LATE_COUNT', 'ADJUSTED_ONTIME_COUNT'
    ]]
)

,DATE,CALENDAR_ID,TRIP_ID,ROUTE_ABBR,TIME_POINT_ABBR,TRIP_EDGE,SCHEDULED_TIME,ACTUAL_DEPARTURE_TIME,ADHERENCE,ADJUSTED_EARLY_COUNT,ADJUSTED_LATE_COUNT,ADJUSTED_ONTIME_COUNT
6,2023-01-01,120230101,332423,50,MCC5_1,1,06:15:00,06:20:12,-5.200000,0,0,1
7,2023-01-01,120230101,332423,50,28&CHARL,0,06:25:00,06:26:55,-1.916666,0,0,1
8,2023-01-01,120230101,332423,50,CH46,0,06:29:00,06:31:29,-2.483333,0,0,1
9,2023-01-01,120230101,332423,50,WHBG,0,06:33:00,06:35:12,-2.200000,0,0,1
10,2023-01-01,120230101,332423,50,HLWD,0,06:40:00,06:40:41,-0.683333,0,0,1
11,2023-01-01,120230101,332423,50,WALM,2,06:47:00,06:44:01,2.983333,0,0,1


Here's an example of another trip later that same day that was considered late at the first time point, departing almost 8 minutes behind schedule.

In [7]:
(
    wego
    .loc[wego['CALENDAR_ID'] == 120230101]
    .loc[wego['TRIP_ID'] == 332493]
    [[
        'DATE', 'CALENDAR_ID', 'TRIP_ID', 'ROUTE_ABBR',
        'TIME_POINT_ABBR', 'TRIP_EDGE',
        'SCHEDULED_TIME', 'ACTUAL_DEPARTURE_TIME', 'ADHERENCE',
        'ADJUSTED_EARLY_COUNT', 'ADJUSTED_LATE_COUNT', 'ADJUSTED_ONTIME_COUNT'
    ]]
)

,DATE,CALENDAR_ID,TRIP_ID,ROUTE_ABBR,TIME_POINT_ABBR,TRIP_EDGE,SCHEDULED_TIME,ACTUAL_DEPARTURE_TIME,ADHERENCE,ADJUSTED_EARLY_COUNT,ADJUSTED_LATE_COUNT,ADJUSTED_ONTIME_COUNT
210,2023-01-01,120230101,332493,50,MCC5_1,1,14:35:00,14:42:44,-7.733333,0,1,0
211,2023-01-01,120230101,332493,50,28&CHARL,0,14:45:00,14:49:29,-4.483333,0,0,1
212,2023-01-01,120230101,332493,50,CH46,0,14:49:00,14:54:25,-5.416666,0,0,1
213,2023-01-01,120230101,332493,50,WHBG,0,14:53:00,14:57:02,-4.033333,0,0,1
214,2023-01-01,120230101,332493,50,HLWD,0,15:01:00,15:03:04,-2.066666,0,0,1
215,2023-01-01,120230101,332493,50,WALM,2,15:10:00,15:06:38,3.366666,0,0,1


This trip departed nearly 5 minutes early from the first time point, so that time point would be considered early.

In [8]:
(
    wego
    .loc[wego['CALENDAR_ID'] == 120230101]
    .loc[wego['TRIP_ID'] == 332431]
    [[
        'DATE', 'CALENDAR_ID', 'TRIP_ID', 'ROUTE_ABBR',
        'TIME_POINT_ABBR', 'TRIP_EDGE',
        'SCHEDULED_TIME', 'ACTUAL_DEPARTURE_TIME', 'ADHERENCE',
        'ADJUSTED_EARLY_COUNT', 'ADJUSTED_LATE_COUNT', 'ADJUSTED_ONTIME_COUNT'
    ]]
)

,DATE,CALENDAR_ID,TRIP_ID,ROUTE_ABBR,TIME_POINT_ABBR,TRIP_EDGE,SCHEDULED_TIME,ACTUAL_DEPARTURE_TIME,ADHERENCE,ADJUSTED_EARLY_COUNT,ADJUSTED_LATE_COUNT,ADJUSTED_ONTIME_COUNT
54,2023-01-01,120230101,332431,50,MCC5_1,1,12:15:00,12:10:04,4.933333,1,0,0
55,2023-01-01,120230101,332431,50,28&CHARL,0,12:25:00,12:27:55,-2.916666,0,0,1
56,2023-01-01,120230101,332431,50,CH46,0,12:29:00,12:30:48,-1.800000,0,0,1
57,2023-01-01,120230101,332431,50,WHBG,0,12:34:00,12:33:37,0.383333,0,0,1
58,2023-01-01,120230101,332431,50,HLWD,0,12:42:00,12:41:42,0.300000,0,0,1
59,2023-01-01,120230101,332431,50,WALM,2,12:50:00,12:48:24,1.600000,0,0,1


**Headway** is the amount of time between a bus and the prior bus at the same stop. In the dataset, the amount of headway scheduled is contained in the SCHEDULED_HDWY column and indicates the difference between the scheduled time for a particular stop and the scheduled time for the previous bus on that same stop.

This dataset contains a column HDWY_DEV, which shows the amount of deviation from the scheduled headway. **Bunching** occurs when there is shorter headway than scheduled, which would appear as a negative HDWY_DEV value. **Gapping** is when there is more headway than scheduled and appears as a positive value in the HDWY_DEV column. Note that you can calculate headway deviation percentage as HDWY_DEV/SCHEDULED_HDWY. 

The generally accepted range of headway deviation is 50% to 150% of the scheduled headway, so if scheduled headway is 10 minutes, a headway deviation of up to 5 minutes would be acceptable (but not ideal).

Here, you can see consecutive trips at the same time point. Notice that the scheduled headway is based on the scheduled time and that the actual headway is based on the actual departure times.

In [9]:
(
    wego
    .loc[wego['ROUTE_DIRECTION_NAME'] == 'TO DOWNTOWN']
    .loc[wego['TIME_POINT_ABBR'] == 'CH46']
    [['DATE', 'TRIP_ID', 'TIME_POINT_ABBR','ROUTE_DIRECTION_NAME', 'TRIP_EDGE', 
      'SCHEDULED_TIME', 'SCHEDULED_HDWY',
      'ACTUAL_DEPARTURE_TIME', 'ACTUAL_HDWY', 'HDWY_DEV'
     ]]
    .sort_values(['DATE', 'SCHEDULED_TIME'])
    .iloc[:5]
)

,DATE,TRIP_ID,TIME_POINT_ABBR,ROUTE_DIRECTION_NAME,TRIP_EDGE,SCHEDULED_TIME,SCHEDULED_HDWY,ACTUAL_DEPARTURE_TIME,ACTUAL_HDWY,HDWY_DEV
3,2023-01-01,332422,CH46,TO DOWNTOWN,0,05:50:00,NaN,05:50:34,NaN,NaN
147,2023-01-01,332482,CH46,TO DOWNTOWN,0,06:20:00,30.0,06:19:11,28.616666,-1.383334
243,2023-01-01,332536,CH46,TO DOWNTOWN,0,06:50:00,30.0,06:50:14,31.050000,1.050000
15,2023-01-01,332424,CH46,TO DOWNTOWN,0,07:20:00,30.0,07:24:18,34.066666,4.066666
159,2023-01-01,332484,CH46,TO DOWNTOWN,0,07:50:00,30.0,07:51:15,26.950000,-3.050000
